# Laboratorio de MLOps - Ciclo de Vida del Flujo de Trabajo de ML
### Víctor Maldonado García
___
#### Objetivo General del Laboratorio
Proporcionar experiencia práctica en la aplicación del ciclo de vida del flujo de trabajo de ML, desde la exploración y preprocesamiento de datos hasta la evaluación y mejora de modelos.

___
### 1° Exploración de datos

Utilizo un conjunto de datos de temblores en Chile que provee el Centro Sismológico Nacional.

In [1]:
import pandas as pd
data = pd.read_csv('seismic_data.csv')
print(data.head(10))

             Date(UTC)  Latitude  Longitude  Depth  Magnitude
0  2024-03-27 05:08:28    -23.54     -68.21    148        5.4
1  2024-03-24 19:50:11    -19.47     -70.08     74        5.1
2  2024-03-13 09:30:06    -18.68     -69.52    114        5.1
3  2024-02-14 01:31:42    -28.91     -71.44     45        5.8
4  2024-02-11 21:14:43    -28.24     -71.32     44        5.7
5  2024-02-03 00:48:55    -32.74     -70.66     92        5.0
6  2024-01-24 08:05:10    -22.17     -68.77    132        5.4
7  2023-12-24 03:23:41    -38.13     -74.16     36        5.4
8  2023-12-21 16:21:52    -21.92     -70.25     55        5.8
9  2023-12-16 22:15:44    -37.05     -74.23     16        4.6


### 2° Preprocesamiento de datos

Se preparan los datos para el modelado, incluyendo la limpieza y la normalización.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Selección de características y variable objetivo
X = data.drop('Date(UTC)', axis=1)
y = data['Magnitude']

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 3° Modelado Básico
Entrenaremos un modelo de regresión lineal para predecir la magnitud de los temblores.

In [4]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_scaled, y_train)

LinearRegression()

## 4° Evaluación de modelos
Evaluaremos el rendimiento del modelo utilizando el conjunto de prueba.

In [5]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'R²: {r2:.2f}')

MSE: 1.847194957129091e-32
R²: 1.00


MSE (Error Cuadrático Medio): El valor del MSE es 1.847194957129091e-32. El MSE mide la diferencia promedio al cuadrado entre las predicciones del modelo y los valores reales en el conjunto de prueba. Un MSE cercano a cero indica que el modelo se ajusta perfectamente a los datos. En este caso, el valor extremadamente pequeño sugiere que las predicciones son muy cercanas a los valores reales.

R² (Coeficiente de Determinación): El valor del R² es 1.00. El R² mide la proporción de la varianza total en la variable objetivo (en este caso, la magnitud) que se explica por el modelo. Un R² de 1.00 significa que el modelo explica el 100% de la variabilidad en la magnitud. En otras palabras, el modelo se ajusta perfectamente a los datos.

In [6]:
# Calculando la varianza y el rango de la variable objetivo
variance = y.var()
data_range = y.max() - y.min()

print(f'MSE: {mse}')
print(f'Varianza de la variable objetivo: {variance}')
print(f'Rango de la variable objetivo: {data_range}')

# Comparando el MSE con la varianza y el rango
print(f'Relación MSE/Varianza: {mse/variance}')
print(f'Relación MSE/Rango: {mse/data_range}')

MSE: 1.847194957129091e-32
Varianza de la variable objetivo: 0.4681168389219627
Rango de la variable objetivo: 6.1000000000000005
Relación MSE/Varianza: 3.9460126266404767e-32
Relación MSE/Rango: 3.0281884543099852e-33


Los resultados indican que el modelo es altamente preciso y se ajusta perfectamente a los datos de prueba.
Sin embargo, hay que tener en cuenta que estos valores extremadamente pequeños pueden ser señales de sobreajuste. Hay que evaluar el modelo en datos no vistos y considerar técnicas como la validación cruzada para obtener estimaciones más robustas del rendimiento.

## 5° Iteración del modelo
Para mejorar el modelo en la iteración, podemos ajustar hiperparámetros del algoritmo actual, como el número de iteraciones en la regresión lineal, o probar diferentes algoritmos que puedan capturar mejor la complejidad de los datos, como Random Forest o Gradient Boosting. También podríamos explorar técnicas avanzadas de preprocesamiento de datos o ingeniería de características para mejorar la precisión del modelo.

A pesar de que el objetivo de este laboratorio es utilizar otro modelo que no fuera Random Forest, lo utilizaré primero para comparar resultados, luego utilizaré otro modelo. 

In [7]:
from sklearn.ensemble import RandomForestRegressor

# Crear y entrenar el modelo de Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Realizar predicciones con el modelo de Random Forest
rf_pred = rf_model.predict(X_test_scaled)

# Evaluar el modelo de Random Forest
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest MSE: {rf_mse}')

Random Forest MSE: 5.180303030303395e-05


Un MSE tan pequeño indica que Random Forest está realizando predicciones precisas en el conjunto de prueba. Sin embargo, como fue mencionado anteriormente, hay que tener en cuenta que valores extremadamente pequeños pueden ser señales de sobreajuste.

#### Regresión Ridge
Ahora para predecir la magnitud de los temblores, un modelo alternativo que probaré es la Regresión Ridge.
El modelo de Regresión Ridge, es una extensión de la regresión lineal que incluye una regularización para evitar el sobreajuste. El parámetro alpha controla el nivel de regularización: valores más altos de alpha aumentan la cantidad de regularización aplicada al modelo.

In [8]:
from sklearn.linear_model import Ridge

# Dividir los datos en conjuntos de entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Crear el modelo de Regresión Ridge
modelo_ridge = Ridge(alpha=1.0)  # Alpha es el parámetro de regularización

# Entrenar el modelo
modelo_ridge.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
predicciones = modelo_ridge.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, predicciones)
print("Error cuadrático medio (MSE) en conjunto de prueba:", mse)


Error cuadrático medio (MSE) en conjunto de prueba: 2.577118711471428e-07


un MSE tan bajo indica que el modelo está funcionando muy bien en términos de ajuste a los datos y capacidad predictiva. Esto es una buena señal de que el modelo de Regresión Ridge está siendo efectivo el problema de predicción de magnitudes de temblores.